# Introducción
Trabajas para la compañía minera OilyGiant. Tu tarea es encontrar el mejor lugar para un nuevo pozo.
Pasos para elegir la ubicación:

- Recolecta los parámetros del pozo de petróleo en la región seleccionada: calidad del petróleo y volumen de reservas.
- Construye un modelo para predecir el volumen de reservas en los nuevos pozos;
- Selecciona los pozos de petróleo con los valores estimados más altos;
- Elige la región con el mayor beneficio total para los pozos de petróleo seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. 

## Objetivo

Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

## Indice

1. Introducción
2. Entrenamiento y prueba del modelo en cada región
3. Cálculo de ganancias
4. Cálculo de ganancias de un conjunto de pozos de petroleo
5. Cálculo de riesgos y ganancias por cada región
6. Conclusiones

## Preparación de los datos

Las fuentes de datos a utilizar se encuentran dentro de la carpeta `datasets` y sus nombres son:

- `geo_data_0`
- `geo_data_1`
- `geo_data_2`

Iniciaremos con la exploración de los datos contenidos en las fuentes anteriormente mencionadas y validar la calidad de los registros.

In [1]:
# Importando librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
# Creación de "df0", "df1", "df2"
df0 = pd.read_csv('datasets/geo_data_0.csv')
df1 = pd.read_csv('datasets/geo_data_1.csv')
df2 = pd.read_csv('datasets/geo_data_2.csv')

In [3]:
# Validando información en "df0"
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
# Visualizando primeros 10 registros de "df0"
df0.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


De manera general, no se visualizan registros nulos o con un tipo de dato incorrecto, por último procederemos con la validación de unicidad de las filas.

In [5]:
# Explorando duplicidad de registros para "df0"
df0.duplicated().sum()

0

El set de datos cuenta con registros únicos, con lo anterior realizado podemos validar que la información con la que se cuenta esta completa para ser procesada. Continuaremos de la misma manera para `df1` y `df2`.

In [6]:
# Validando información en "df1"
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
# Visualizando primeros 10 registros de "df1"
df1.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [8]:
# Explorando duplicidad de registros para "df1"
df1.duplicated().sum()

0

In [9]:
# Validando información en "df2"
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
# Visualizando primeros 10 registros de "df2"
df2.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [11]:
# Explorando duplicidad de registros para "df2"
df2.duplicated().sum()

0

Una vez verificando que los registros se encuentran en orden y listos para ser procesados crearemos el modelo de entrenamiento bajo la siguinte característica:

- El conjunto de entrenamiento y el conjunto de validación tendrán una proporción de 75:25.

# Entrenamiento y prueba de modelo para cada región

Con intención de no ejecutar procesamiento innecesario, uno de forma intuitiva crearía un solo modelo de `regresión líneal` para entrenar los datos de las 3 regiones proporcionadas, sin embargo, debido a la poca información que se tiene de las caracteríasticas y el desconocimiento del como influyen en las ganancias que generan los pozos, podría ser conveniente primero realizar el entrenamiento de modelos disintos con su respectiva región y hacer pruebas con los conjuntos de pruebas del resto de regiones para evaluar el rendimiento de predicción y poder concretar la forma de trabajo.

## Dividiendo el conjunto de datos en el conjunto de entrenamiento y el conjunto de validación para la región "df0"

In [12]:
# Dividiendo el conjunto "df0" en conjunto de entrenamiento y conjunto de validación
X = df0.drop(['id','product'],axis=1)
y = df0['product']

features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(X,y,test_size=0.25,random_state=12345)


In [13]:
# Creando el modelo de regresión líneal para "df0"
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)

LinearRegression()

## Dividiendo el conjunto de datos en el conjunto de entrenamiento y el conjunto de validación para la región "df1"

In [14]:
# Dividiendo el conjunto "df1" en conjunto de entrenamiento y conjunto de validación
X = df1.drop(['id','product'],axis=1)
y = df1['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(X,y,test_size=0.25,random_state=12345)


In [15]:
# Creando el modelo de regresión líneal para "df1"
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)

LinearRegression()

## Dividiendo el conjunto de datos en el conjunto de entrenamiento y el conjunto de validación para la región "df2"

In [16]:
# Dividiendo el conjunto "df2" en conjunto de entrenamiento y conjunto de validación
X = df2.drop(['id','product'],axis=1)
y = df2['product']

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(X,y,test_size=0.25,random_state=12345)


In [17]:
# Creando el modelo de regresión líneal para "df2"
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

LinearRegression()

Una vez teniendo los modelos entrenados, crearemos una especie de `matriz de confusión` en la cuál asignaremos el `score` obtenido al hacer predicciones con los diferentes conuntos de validación.

In [18]:
scores = {'model_0': [model_0.score(features_valid_0,target_valid_0)],
          'model_1': [model_1.score(features_valid_1,target_valid_1)],
          'model_2': [model_2.score(features_valid_2,target_valid_2)]}

pd.DataFrame(data=scores)

,model_0,model_1,model_2
0,0.279943,0.999623,0.205248


Lo que podemos obsevar en el resultado es que tenemos un coeficiente de determinación distinto para cada modelo, es decir, la recta que se logra modelar a través de la `regresión linal`, tiene diferentes pendientes ademas que tenemos un modelo que es una recta completamente horizontal

Para los diferentes casos, podemos observar:
- `modelo_0`: Genera una recta levemente positiva.
- `modelo_1`: Genera una recta completamente horizontal.
- `modelo_2`: Generea una recta levemente negativa.

Lo que nos lleva a concluir que cada uno de los set de datos debe ser trabajado de forma aislada del resto puesto que sus características no ponderan de la misma forma en todos los casos. Por tal motivo, realizaremos las predicciones para cada set de datos para posteriormente analizar los resultados y evaluar las rectas modeladas.

In [19]:
# Guardando los resultados de las predicciones
result_0 = model_0.predict(features_valid_0)
result_1 = model_1.predict(features_valid_1)
result_2 = model_2.predict(features_valid_2)

Una vez teniendo los valores de las predicciones, una forma de poder evaluar la recta generada por el modelo es haciendo uso de `RECM` ó también conocida como `Raíz del error cuadrático medio`, que es una medida que nos permite calcular la diferencia entre los valores predichos por un modelo y los valores observados. En pocas palabras nos va a permitir encontrar la diferencia (distancia) entre la posición de la recta y los valores reales.

## Calculando la "Raíz del error cuadrático medio"

In [20]:
recm = {'RECM_0': [mean_squared_error(result_0,target_valid_0,squared=False)],
        'RECM_1': [mean_squared_error(result_1,target_valid_1,squared=False)],
        'RECM_2': [mean_squared_error(result_2,target_valid_2,squared=False)]}

pd.DataFrame(data=recm,index=['Distancia promedio'])

,RECM_0,RECM_1,RECM_2
Distancia promedio,37.579422,0.893099,40.029709


Como podemos ver en la tabla, y como era de esperarse, al tratarse de `model_0` podemos ver que hay una diferencia promedio de aproximadamente `38` unidades entre los valores reales y la recta modelada. Respecto a `model_1` vemos que hay una diferencia promedio practicamente `nula (0.89)` entre los valores reales y la recta modelada. Por último para `model_2` hay una diferencia promedio de `40` unidades aproximadamente entre la recta y los valores reales.

# Cálculo de ganancias

El presupuesto que se tiene para el desarrollo de 200 pozos petroleros es de `100 millones de dolares`, es decir, un aproximado de `500 mil dolares` por pozo aproximadamente. Por lo que si un barril de materia prima genera `4.5 dolares` de ingreso, el pozo debe superar al menos la inversión realizada. Por otro lado, el volumen de reservas esta expresado en miles de barriles. Por estas razones, debemos calcular la cantidad de reserva mínima suficiente para desarrollar un nuevo pozo sin perdidas.


## Calculando el volumen de reservas suficientes para no tener perdidas

Para poder hacer este cálculo, debemos basarnos en la información que anteriormente se porporciono:
- No debemos tener perdidas, es decir, la ganancia mínima que debe tener cada pozo debe ser de `500 mil`.
- La ganancia por cada mil barriles es de `4500 dolares`.

Con esta información tendremos una formula de la siguiente manera:


$$500,000 = (4,500) (barriles)$$


Despejando `# barriles` obtenemos:


$$barriles = \frac{500,000}{4,500}$$


Por lo que para que un pozo no tenga perdidas, al menos debe contar con un total de `111.111 mil` barriles.

In [21]:
# Comparando la cifra mínima con la media de todas las regiones
means = {'Media df0': df0['product'].mean(),
         'Media df1': df1['product'].mean(),
         'Media df2': df2['product'].mean()}

pd.DataFrame(data=means,index=['111.111'])

,Media df0,Media df1,Media df2
111.111,92.5,68.825,95.0


Como podemos ver, la reserva media de cada región se encuentra por debajo de la cantidad necesaria para no tener perdidas en la inversión, sin embargo una posible solución es que, si se tuviera más información, las regiones podrían ser acotadas aun más de tal modo que se pudieran tomar unicaménte datos de aquellos pozos que tienen reservas grandes de petroleo.

Sin embargo como no tenemos información suficientemente explicita para poder hacer dicha acotación, es importante seguir trabajando con los datos para poder hacer un análisis completo por regiones.

# Cálculo de ganancias de un conjunto de pozos de petroleo

Una vez conociendo la cantidad mínima de reserva para no tener perdidas, crearemos una función que permita obtener el ingreso al extraer las reservas. Para esto, por regiones, extraeremos los pozos con mayor margen de producto y sobre estos empezaremos a trabajar.

## 200 pozos más rentables predichos por el modelo para las distintas regiones

Comenzaremos con concatenar los conjuntos de validación de las distintas regiones con las predicciones realizadas por el modelo. Esto se hace con la intención de poder tener una comparativa entre el valor real y la predicción pues hay variaciones que se encuentran muy alejadas de los valores reales y por tal motivo es importante asegurarse que aquellos pozos a seleccionar para el cálculo de ganancias sean las predicciones con mejor calidad.

In [22]:
# Concatenacion del conjunto de validación y las predicciones para la región "0"
res0 = pd.concat([features_valid_0,target_valid_0],axis=1)
res0.reset_index(drop=True,inplace=True)
res0 = pd.concat([res0,pd.Series(result_0,name='result')],axis=1)
res0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       25000 non-null  float64
 1   f1       25000 non-null  float64
 2   f2       25000 non-null  float64
 3   product  25000 non-null  float64
 4   result   25000 non-null  float64
dtypes: float64(5)
memory usage: 976.7 KB


In [23]:
# Concatenacion del conjunto de validación y las predicciones para la región "1"
res1 = pd.concat([features_valid_1,target_valid_1],axis=1)
res1.reset_index(drop=True,inplace=True)
res1 = pd.concat([res1,pd.Series(result_1,name='result')],axis=1)
res1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       25000 non-null  float64
 1   f1       25000 non-null  float64
 2   f2       25000 non-null  float64
 3   product  25000 non-null  float64
 4   result   25000 non-null  float64
dtypes: float64(5)
memory usage: 976.7 KB


In [24]:
# Concatenacion del conjunto de validación y las predicciones para la región "2"
res2 = pd.concat([features_valid_2,target_valid_2],axis=1)
res2.reset_index(drop=True,inplace=True)
res2 = pd.concat([res2,pd.Series(result_2,name='result')],axis=1)
res2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       25000 non-null  float64
 1   f1       25000 non-null  float64
 2   f2       25000 non-null  float64
 3   product  25000 non-null  float64
 4   result   25000 non-null  float64
dtypes: float64(5)
memory usage: 976.7 KB


In [25]:
# Creación de la función "income"
def income(data):
    total_income = data.sort_values(by='result',ascending=False)
    total_income = total_income['product'].head(200).sum()
    return total_income*4_500 - 100_000_000

Teniendo las funciones a utilizar podemos determinar cuál sería el margén de ganancia por región una vez se presentan las predicciones realizadas con mayor ingreso.

Las funciones sirven para lo siguiente:

- `values`: Identifica las predicciones que están de al menos `111.111 mil` barriles de reserva y que no se encuentran tan alejadas de los valores reales al cuál se le pasa el `set dedatos` con el que se va a trabajar.
- `income`: Calcula el beneficio neto (ingreso menos inversión por falta de más datos) a la cuál se le pasan dos parámetros, el primero es el `set de datos` con el que se va a trabajar y el segundo es el `tamaño del bloque` a considerar.

In [26]:
# Aplicando la función "income"
income_0 = income(res0)
income_1 = income(res1)
income_2 = income(res2)

In [27]:
# Cuadro comparativo de "ingresos" de los 200 pozos con mayor cantidad de reserva de petroleo en barriles
data_income = {'Zona 0': [str(income_0)],
               'Zona 1': [str(income_1)],
               'Zona 2': [str(income_2)]}
pd.DataFrame(data=data_income,index=['Beneficio neto'])

,Zona 0,Zona 1,Zona 2
Beneficio neto,33208260.43139851,24150866.966815114,27103499.635998324


Como podemos observar en la tabla, el beneficio neto de los `200` pozos mejor predichos de cada zona es el siguiente:

- `Zona 0`: Tiene un beneficio neto de `39,960,488 USD`, se decir, a los `ingresos` se le resto la inversión de `100,000,000 USD`.
- `Zona 1`: Tiene un beneficio neto de `24,857,120 USD`, que aunque el modelado de la recta se encontraba considerablemente mejor ajustado, vemos que es menor que el caso de la `Zona 0`.
- `Zona 2`: Tiene un beneficio neto de `33,217,543 USD` que si bien es más próximo a la `Zona 0` aun se encuentra por debajo de la misma.

En conclusión, de acuerdo con los datos y las predicciones realizadas por el modelo de `regresión lineal`, la `Zona 0` es la zona con mejor `beneficio neto` para `OilyGiant` pues puede obtener el retorno de inversión más una ganancia liquida al hacer uso de estos pozos y sus aledaños pues cotienen la mayor cantidad de reservas de petroleo (medido en barriles) que el resto de las zonas.

# Cálculo de riesgos y ganancias por cada región

Haciendo un cálculo del `beneficio neto` no es suficiente para considerar una inversión segura (en el sentido de maximizar las ganancias) pues es importante evaluar los riesgos que esto implica y que pueden ser diversos, sin embargo, y en este caso en particular, como hemos notado hasta el momento, la cantidad de información es limitada y poco explicita por lo que no contamos con información suficiente para usar técnicas tradicionales de calculo de riesgos y beneficios, sin embargo, para este tipo de casos podemos aplicar la técnica del `bootstraping`.

El `bootstraping` es una técnica que permite a través de información limitada poder hacer cálculos aplicados a los negocios que nos permitan conocer sus riesgos, beneficios, etc. Por esta razón haremos uso de esta técnica aplicada a `1000` muestras para encontrar la distribución del beneficio con un `99%` de probabilidad de que sea un pozo prometedor.

## Función de Bootstraping

In [28]:
# Función de boostraping
def bootstraping(data):
    subsample_res = []
    state = np.random.RandomState(12345)
    for _ in range(1000):
        subsample = data.sample(n=500,replace=True,random_state=state)
        subsample_res.append(income(subsample)) # .quantile(q=0.99)
    return subsample_res

Esta función retorna una arreglo que contiene las medias de las muestras generadas, por lo que puede ser trabajadas de distintas formas sin embargo lo que nos interesa conocer es sobre que rangos es que encuentra cada zona por lo que vamos a encontrarlo.

## Aplicando "Bootstraping" y cálculando el intrevalo de confianza para la región "0"

OilyGiant solicita que el intervalo de confianza sea del `95%` por lo que este se dividirá en dos partes, el límite inferior de `2.5%` y el límite superior de `97.5%`.

In [29]:
# Aplicando Bootstraping
interval_0 = pd.Series(bootstraping(res0))
lower_0 = interval_0.quantile(0.025)
upper_0 = interval_0.quantile(0.975)
print('Límite inferior:',lower_0)
print('Límite superior:',upper_0)

Límite inferior: -1112155.4589049604
Límite superior: 9097669.41553423


Como podemos observar, nos encontramos con un intervalo de confianza satisfactorio en el que se tiene un margen de ganancia sobrepasando la reserva mínima encontrada.

## Aplicando "Bootstraping" y cálculando el intrevalo de confianza para la región "1"

In [30]:
# Aplicando Bootstraping
interval_1 = pd.Series(bootstraping(res1))
lower_1 = interval_1.quantile(0.025)
upper_1 = interval_1.quantile(0.975)
print('Límite inferior:',lower_1)
print('Límite superior:',upper_1)

Límite inferior: 338205.0939898458
Límite superior: 8522894.538660347


## Aplicando "Bootstraping" y cálculando el intrevalo de confianza para la región "2"

In [31]:
# Encontrando el intrevalo de confianza
interval_2 = pd.Series(bootstraping(res2))
lower_2 = interval_2.quantile(0.025)
upper_2 = interval_2.quantile(0.975)
print('Límite inferior:',lower_2)
print('Límite superior:',upper_2)

Límite inferior: -1633504.1339559986
Límite superior: 9503595.749237997


De acuerdo con los resultados, podemos obsevar que los intervalos de confianza soy muy reducidos, es decir, que varían decimas, sin embargo debemos tomar en cuenta que estos rangos estan hablando de `miles de barriles`, por lo que es importante hacer las evaluaciones respectivas tomando en cuenta estos decimales.

## Cálculo de perdidas

Una vez que conocemos el intervalo de confianza de las distintas zonas, y podemos calcular también el beneficio promedio de cada una de ellas, calcularemos la pérdida expresada como una probabilidad y posteriormente como un porcentaje.

In [33]:
# Calculo de perdida en "probabilidades"
p0 = (interval_0 < 0).mean()
p1 = (interval_1 < 0).mean()
p2 = (interval_2 < 0).mean()

Una vez teniendo la información, procederemos a mostrarla.

In [34]:
# Información obtenida
data_end = {'Beneficio promedio': [res0['result'].mean(),
                                   res1['result'].mean(),
                                   res2['result'].mean()],
            'Intervalo inferior': [lower_0,lower_1,lower_2],
            'Intervalo superior': [upper_0,upper_1,upper_2],
            'Porcentaje de riesgo': [p0*100,p1*100,p2*100]}

pd.DataFrame(data=data_end,index=['Zona 0','Zona 1','Zona 2'])


,Beneficio promedio,Intervalo inferior,Intervalo superior,Porcentaje de riesgo
Zona 0,92.592568,-1.112155e+06,9.097669e+06,6.9
Zona 1,68.728547,3.382051e+05,8.522895e+06,1.5
Zona 2,94.965046,-1.633504e+06,9.503596e+06,7.6


# Conclusiones

De acuerdo con la información propocionada por los resultados podemos observar que tanto la `Zona 0` como la `Zona 2` tiene un riesgo más elevado que la `Zona 1` esto puede ser marcado por distintas situaciones destacando de entre ellas las siguientes:

- El modelo de regresión lineal generado para hacer las predicciones de los resultados, se ajusta mucho mejor de tal manera que hay una variación máxima de `una unidad aproximadamente` entre las predicciones y los valores reales.

- Por otro lado podemos observar que el intervalo de confianza que esta zona proporciona se mantiene por encima de `0`, es decir, no tiene valores negativos lo que aumenta la confianza en esta zona.

En conclusión podemos decir que la `Zona 1` es la zona que mejor se ajusta a las necesidades de la inversión pues máximiza el rendimiento que se puede generar sin perder de vista el riesgo que esto implica que es del `1.5%` lo cuál genera confianza a "OilyGiant" para invertir.